In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import nltk
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
import pickle
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [2]:
pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 41.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
df = pd.read_csv("sms.csv")

In [5]:
df.sample(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
3551,ham,Lol u still feeling sick?,NaN,NaN,NaN
4997,ham,Can you talk with me..,NaN,NaN,NaN
3166,ham,"When people see my msgs, They think Iam addict...",NaN,NaN,NaN
1430,ham,Don't look back at the building because you ha...,NaN,NaN,NaN
5342,spam,u r subscribed 2 TEXTCOMP 250 wkly comp. 1st w...,NaN,NaN,NaN
4278,spam,URGENT. Important information for 02 user. Tod...,NaN,NaN,NaN
3959,ham,I sent lanre fakeye's Eckankar details to the ...,NaN,NaN,NaN
199,ham,"Found it, ENC &lt;#&gt; , where you at?",NaN,NaN,NaN
725,ham,Got it! It looks scrumptious... daddy wants to...,NaN,NaN,NaN
3397,ham,It only does simple arithmetic not percentages.,NaN,NaN,NaN


In [6]:
df.shape

(5572, 5)

Data Cleaning

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [8]:
#dropping the last three columns
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [9]:
df.sample(5)

,v1,v2
1545,ham,"Good afternoon, my love ! Any job prospects ? ..."
5194,ham,Ok... Take ur time n enjoy ur dinner...
1299,ham,Great to hear you are settling well. So what's...
5286,ham,I'm still pretty weak today .. Bad day ?
451,ham,Thanks for this hope you had a good day today


In [10]:
encoder = LabelEncoder()

In [11]:
df['v1'] = encoder.fit_transform(df['v1'])

In [12]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df.isnull().sum()  #missing values

v1    0
v2    0
dtype: int64

In [14]:
df.duplicated().sum()  #duplicate values

409

In [15]:
df = df.drop_duplicates(keep = 'first')

In [16]:
df.shape

(5163, 2)

EDA

In [17]:
df['v1'].value_counts()

0    4516
1     647
Name: v1, dtype: int64

In [18]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
df['num_characters'] = df['v2'].apply(len)

In [20]:
df['num_words'] = df['v2'].apply(lambda x:len(nltk.word_tokenize(x)))

In [21]:
df['num_sentences'] = df['v2'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [22]:
df.head()

,v1,v2,num_characters,num_words,num_sentences
0,0,"Go until jurong point, crazy.. Available only ...",111,24,2
1,0,Ok lar... Joking wif u oni...,29,8,2
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37,2
3,0,U dun say so early hor... U c already then say...,49,13,1
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15,1


In [23]:
df[['num_characters', 'num_words', 'num_sentences']].describe()

,num_characters,num_words,num_sentences
count,5163.000000,5163.000000,5163.000000
mean,78.811156,18.444122,1.963974
std,58.153491,13.327940,1.448008
min,2.000000,1.000000,1.000000
25%,36.000000,9.000000,1.000000
50%,60.000000,15.000000,1.000000
75%,116.000000,26.000000,2.000000
max,910.000000,220.000000,38.000000


In [24]:
#ham
df[df['v1'] == 0][['num_characters', 'num_words', 'num_sentences']].describe()

,num_characters,num_words,num_sentences
count,4516.000000,4516.000000,4516.000000
mean,70.428919,17.123782,1.820195
std,56.356310,13.493970,1.383657
min,2.000000,1.000000,1.000000
25%,34.000000,8.000000,1.000000
50%,52.000000,13.000000,1.000000
75%,90.000000,22.000000,2.000000
max,910.000000,220.000000,38.000000


In [25]:
#spam
df[df['v1'] == 1][['num_characters', 'num_words', 'num_sentences']].describe()

,num_characters,num_words,num_sentences
count,647.000000,647.000000,647.000000
mean,137.318393,27.659969,2.967543
std,30.096861,7.038065,1.490069
min,13.000000,2.000000,1.000000
25%,130.000000,25.000000,2.000000
50%,148.000000,29.000000,3.000000
75%,157.000000,32.000000,4.000000
max,223.000000,46.000000,9.000000


In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
def transform_sms(sms):
    sms = sms.lower()
    sms = nltk.word_tokenize(sms)
    
    y = []
    for i in sms:
        if i.isalnum():
            y.append(i)
    
    sms = y[:]
    y.clear()
    
    for i in sms:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
   
    #return y
    return " ".join(y)

In [28]:
transform_sms("FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, ï¿½1.50 to rcv")

'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send rcv'

In [29]:
transform_sms(df['v2'][2])

'free entry 2 wkly comp win fa cup final tkts 21st may text fa 87121 receive entry question std txt rate c apply 08452810075over18'

In [30]:
df['v2'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [31]:
df['transformed_sms'] = df['v2'].apply(transform_sms) 

In [32]:
df.sample(5)

,v1,v2,num_characters,num_words,num_sentences,transformed_sms
1912,0,For real tho this sucks. I can't even cook my ...,87,22,3,real tho sucks ca even cook whole electricity ...
5278,1,URGENT! Your Mobile number has been awarded wi...,143,27,5,urgent mobile number awarded prize guaranteed ...
2937,0,Hey i've booked the pilates and yoga lesson al...,59,12,1,hey booked pilates yoga lesson already haha
2142,0,You see the requirements please,31,5,1,see requirements please
3889,1,Double Mins & 1000 txts on Orange tariffs. Lat...,151,25,3,double mins 1000 txts orange tariffs latest mo...


Model Building

In [46]:
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features = 3000)

In [47]:
X = tfidf.fit_transform(df['transformed_sms']).toarray()

In [48]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
X.shape

(5163, 3000)

In [50]:
Y = df['v1'].values

In [51]:
Y

array([0, 0, 1, ..., 0, 0, 0])

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)

In [53]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [54]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [55]:
gnb.fit(X_train, Y_train)
Y_pred1 = gnb.predict(X_test)
print(accuracy_score(Y_test,Y_pred1))
print(confusion_matrix(Y_test,Y_pred1))
print(precision_score(Y_test,Y_pred1))

0.8644724104549855
[[788 124]
 [ 16 105]]
0.4585152838427948


In [56]:
mnb.fit(X_train, Y_train)
Y_pred2 = mnb.predict(X_test)
print(accuracy_score(Y_test,Y_pred2))
print(confusion_matrix(Y_test,Y_pred2))
print(precision_score(Y_test,Y_pred2))

0.9757986447241046
[[911   1]
 [ 24  97]]
0.9897959183673469


In [57]:
bnb.fit(X_train, Y_train)
Y_pred3 = bnb.predict(X_test)
print(accuracy_score(Y_test,Y_pred3))
print(confusion_matrix(Y_test,Y_pred3))
print(precision_score(Y_test,Y_pred3))

0.9825750242013552
[[910   2]
 [ 16 105]]
0.9813084112149533


In [59]:
pickle.dump(tfidf, open('vector.pkl', 'wb'))
pickle.dump(mnb, open('model.pkl', 'wb'))           
            